In [ ]:
WITH InitialSales AS (
    SELECT  
    a.cIDCompany Company_DwID
    ,a.cIDInvoice Invoice_DwID
    ,i.cSalesPoint
    ,a.cSerie Serie
    ,MC1_GDescription.cDescription Form
    ,a.cIDCustomer Customer_DwID 
    ,a.cIDBranchInvoice BranchDwID
    ,a.cIDTerritory Territory_DwID
    ,a.dEmission Emission
    ,a.xChannelDistribution ChannelDistribution 
    ,a.xReturnReason InvoiceCancelReason
    ,b.cidOrder Order_DwID
    ,a.xInvoiceType InvoiceType 
    ,e.cIDProduct Product_DwID
    ,e.nAmount Amount
    ,e.nUnitValue UnitValue
    ,e.nTotalValue TotalValue
    ,e.MC1LastUpdate
    ,e.nFreegoods Freegoods
    ,g.nSeq Seq
    ,g.nValue Value
    ,g.cIDPricing Pricing_DwID
    ,g.cIDStep Step_DwID
    ,g.npercentage
    ,f.xIDReasonReturnStock ReasonReturnStock_DwID
    ,h.lCalculated Calculated
    ,h.cIDPricingFormationRule PricingFormationRule_DwID
    from CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_Invoice a
    --################# order invoice #########################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_OrderInvoice 
    ) b 
    on a.cIDCompany = b.cIDCompany
    and a.cIDInvoice = b.cIDInvoice
    and a.cForm = b.cform
    and a.cSerie = b.cSerie
    and a.cIDBranchInvoice =b.cIDBranchInvoice
    AND a.mc1Enabled = 1
    and b.mc1enabled = 1
    --################## invoice item #####################
    inner join (select *
    FROM CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceItem
    ) e 
    on e.cIDCompany = a.cIDCompany 
    and e.cIDInvoice = a.cIDInvoice 
    and e.cForm = a.cform
    and e.cSerie = a.cSerie
    AND e.cIDBranchInvoice=a.cIDBranchInvoice 
    and e.mc1enabled = 1
    --################ invoice item detail #######################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceItemDetail 
    ) g
    on g.cIDCompany = e.cIDCompany
    and g.cIDInvoice = e.cIDInvoice
    and g.cForm = e.cform
    and g.cSerie = e.cSerie
    AND g.cIDBranchInvoice = e.cIDBranchInvoice
    and g.nSeq = e.nSeq --igualando pasos 
    and g.cidProduct = e.cidProduct --igualando los productos de factura y order
    and g.mc1enabled = 1
    --and g.cIDStep in ('00100','00130','00160','00200','00250','00300','00400','00450','00500','00600','00700','00800','00900','01400') --steps para llegar a la venta neta 
    and g.cIDStep in ('00800','00810','00815','00817','00818','00820','00825','00830','00835','00840','00845','00850','00855','00860','00865','00870','00875','00880','00885','00890','00895','00900','00905','00910','00915','00920','07500','07800') --steps de impuestos
    --################### MC1_MC_OrderExt ########################
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_OrderExt
    ) f 
    on f.cIDOrder = b.cIDOrder
    and f.cIDCompany = b.cIDCompany
    and f.mc1enabled = 1
    --#################### REQUEST EMITES ######################
    INNER JOIN (SELECT * 
    FROM CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceRequestEmites
    ) m
    on a.cIDCompany = m.cIDCompany
    and a.cIDInvoice = m.cIDInvoice
    and a.cForm = m.cform
    and a.cSerie = m.cSerie
    and a.cIDBranchInvoice =m.cIDBranchInvoice
    and m.mc1enabled = 1
    and UPPER(m.xInvoiceRequestStatus)= 'SUCCEEDED' 
    --################### PRICING_RULE ###################
    inner join (
    select *
    from CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_PricingFormationRule
    ) h
    on g.cIDStep = h.cIDStep
    and g.cIDPricing = h.cIDPricing
    and g.cIDCompany = h.cIDCompany
    and g.mc1enabled = 1
    and h.mc1enabled = 1
    and upper(h.cIDPricingFormationRule) not in ('UNITVALUECALCULATED','IVATAXVALUE') --se repite el paso 00100 y parece aumentar el valor 
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceExt 
    ) i 
    on a.cIDCompany = i.cIDCompany
    and a.cIDInvoice = i.cIDInvoice
    and a.cForm = i.cform
    and a.cSerie = i.cSerie
    and a.cIDBranchInvoice =i.cIDBranchInvoice
    AND i.mc1Enabled = 1
    left join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_GeneralDescription MC1_GDescription
    on a.cForm = MC1_GDescription.cIDDomainType
    and MC1_GDescription.cDomainType = 'cForm'
    where a.xInvoiceType not in ('ZOGRD','ZOGR') --Ordertype no buscados a fin de ver os nuevos
    and format(a.dEmission , 'yyyy-MM-dd') between '2024-03-16' and '2024-03-16' --informacion de un dia  
),
FirstDeduplicates AS(
    SELECT
        *
    FROM (
        SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION BY a.BranchDwID, a.ChannelDistribution, 
            a.Company_DwID, a.Customer_DwID, a.Emission, a.Form, a.FreeGoods, a.Invoice_DwID, a.InvoiceCancelReason, a.InvoiceType,
            a.Pricing_DwID, a.Product_DwID, a.Seq, a.Serie, a.Step_DwID, a.Territory_DwID ORDER BY a.mc1LastUpdate DESC) as rn
    FROM InitialSales a
    ) sub
    WHERE rn = 1
),
Transformations AS (
    SELECT
        a.Company_DwID,
        a.Invoice_DwID,
        a.cSalesPoint,
        a.Serie,
        a.Form,
        a.Customer_DwID,
        a.BranchDwID,
        a.Territory_DwID,
        format(a.Emission, 'yyyy-MM-dd') as Emission,
        a.ChannelDistribution,
        a.InvoiceCancelReason,
        a.Order_DwID,
        a.InvoiceType,
        a.Product_DwID,
        a.Amount,
        a.UnitValue,
        a.TotalValue,
        a.MC1LastUpdate,
        a.Freegoods,
        a.Seq,
        a.Value,
        a.Pricing_DwID,
        a.Step_DwID,
        a.npercentage,
        a.ReasonReturnStock_DwID,
        a.Calculated,
        a.PricingFormationRule_DwID,
        CASE WHEN a.Step_DwID = '00800' THEN a.Value ELSE 0 END AS impuesto_IVA,
        CASE WHEN a.Step_DwID = '00810' THEN a.Value ELSE 0 END AS impuesto_IVAS,
        CASE WHEN a.Step_DwID = '00815' THEN a.Value ELSE 0 END AS impuesto_PIVA,
        CASE WHEN a.Step_DwID = '00817' THEN a.Value ELSE 0 END AS impuesto_PELR,
        CASE WHEN a.Step_DwID = '00818' THEN a.Value ELSE 0 END AS impuesto_PERN,
        CASE WHEN a.Step_DwID = '00820' THEN a.Value ELSE 0 END AS impuesto_PEMI,
        CASE WHEN a.Step_DwID = '00825' THEN a.Value ELSE 0 END AS impuesto_PECH,
        CASE WHEN a.Step_DwID = '00830' THEN a.Value ELSE 0 END AS impuesto_PEER,
        CASE WHEN a.Step_DwID = '00835' THEN a.Value ELSE 0 END AS impuesto_PEJY,
        CASE WHEN a.Step_DwID = '00840' THEN a.Value ELSE 0 END AS impuesto_PELP,
        CASE WHEN a.Step_DwID = '00845' THEN a.Value ELSE 0 END AS impuesto_PESJ,
        CASE WHEN a.Step_DwID = '00850' THEN a.Value ELSE 0 END AS impuesto_PETF,
        CASE WHEN a.Step_DwID = '00855' THEN a.Value ELSE 0 END AS impuesto_PEBA,
        CASE WHEN a.Step_DwID = '00860' THEN a.Value ELSE 0 END AS impuesto_PECA,
        CASE WHEN a.Step_DwID = '00865' THEN a.Value ELSE 0 END AS impuesto_PECB,
        CASE WHEN a.Step_DwID = '00870' THEN a.Value ELSE 0 END AS impuesto_PECR,
        CASE WHEN a.Step_DwID = '00875' THEN a.Value ELSE 0 END AS impuesto_PECF,
        CASE WHEN a.Step_DwID = '00880' THEN a.Value ELSE 0 END AS impuesto_PENQ,
        CASE WHEN a.Step_DwID = '00885' THEN a.Value ELSE 0 END AS impuesto_PESF,
        CASE WHEN a.Step_DwID = '00890' THEN a.Value ELSE 0 END AS impuesto_PESE,
        CASE WHEN a.Step_DwID = '00895' THEN a.Value ELSE 0 END AS impuesto_PEMZ,
        CASE WHEN a.Step_DwID = '00900' THEN a.Value ELSE 0 END AS impuesto_PESL,
        CASE WHEN a.Step_DwID = '00910' THEN a.Value ELSE 0 END AS impuesto_PESA,
        CASE WHEN a.Step_DwID = '00920' THEN a.Value ELSE 0 END AS impuesto_PETU,
        CASE WHEN a.Step_DwID = '07800' THEN a.Value ELSE 0 END AS Venta_Impuestos,
        CASE WHEN a.Step_DwID = '07500' THEN a.Value ELSE 0 END AS Impuestos_Total
    FROM FirstDeduplicates a
),
GroupedSales AS (
    SELECT
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        npercentage,
        Order_DwID,
        Pricing_DwID,
        PricingFormationRule_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Step_DwID,
        Territory_DwID,
        UnitValue,
        SUM(impuesto_IVAS) AS impuesto_IVAS,
        SUM(impuesto_PIVA) AS impuesto_PIVA,
        SUM(impuesto_PEMI) AS impuesto_PEMI,
        SUM(impuesto_PECH) AS impuesto_PECH,
        SUM(impuesto_PEER) AS impuesto_PEER,
        SUM(impuesto_PEJY) AS impuesto_PEJY,
        SUM(impuesto_PELP) AS impuesto_PELP,
        SUM(impuesto_PESJ) AS impuesto_PESJ,
        SUM(impuesto_PETF) AS impuesto_PETF,
        SUM(impuesto_PEBA) AS impuesto_PEBA,
        SUM(impuesto_PECA) AS impuesto_PECA,
        SUM(impuesto_PECB) AS impuesto_PECB,
        SUM(impuesto_PECR) AS impuesto_PECR,
        SUM(impuesto_PECF) AS impuesto_PECF,
        SUM(impuesto_PENQ) AS impuesto_PENQ,
        SUM(impuesto_PESF) AS impuesto_PESF,
        SUM(impuesto_PESE) AS impuesto_PESE,
        SUM(impuesto_PEMZ) AS impuesto_PEMZ,
        SUM(impuesto_PESL) AS impuesto_PESL,
        SUM(impuesto_PESA) AS impuesto_PESA,
        SUM(impuesto_PETU) AS impuesto_PETU,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(impuesto_IVA) AS impuesto_IVA,
        SUM(Impuestos_Total) AS Impuestos_Total,
        SUM(impuesto_PELR) AS impuesto_PELR,
        SUM(impuesto_PERN) AS impuesto_PERN
    FROM Transformations
    GROUP BY
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        npercentage,
        Order_DwID,
        Pricing_DwID,
        PricingFormationRule_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Step_DwID,
        Territory_DwID,
        UnitValue
),
GroupesSales2 AS (
      SELECT
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        FreeGoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        npercentage,
        Pricing_DwID,
        PricingFormationRule_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        Step_DwID,
        Territory_DwID,
        UnitValue,
        SUM(Amount) AS Amount,
        SUM(impuesto_IVAS) AS impuesto_IVAS,
        SUM(impuesto_PIVA) AS impuesto_PIVA,
        SUM(impuesto_PEMI) AS impuesto_PEMI,
        SUM(impuesto_PECH) AS impuesto_PECH,
        SUM(impuesto_PEER) AS impuesto_PEER,
        SUM(impuesto_PEJY) AS impuesto_PEJY,
        SUM(impuesto_PELP) AS impuesto_PELP,
        SUM(impuesto_PESJ) AS impuesto_PESJ,
        SUM(impuesto_PETF) AS impuesto_PETF,
        SUM(impuesto_PEBA) AS impuesto_PEBA,
        SUM(impuesto_PECA) AS impuesto_PECA,
        SUM(impuesto_PECB) AS impuesto_PECB,
        SUM(impuesto_PECR) AS impuesto_PECR,
        SUM(impuesto_PECF) AS impuesto_PECF,
        SUM(impuesto_PENQ) AS impuesto_PENQ,
        SUM(impuesto_PESF) AS impuesto_PESF,
        SUM(impuesto_PESE) AS impuesto_PESE,
        SUM(impuesto_PEMZ) AS impuesto_PEMZ,
        SUM(impuesto_PESL) AS impuesto_PESL,
        SUM(impuesto_PESA) AS impuesto_PESA,
        SUM(impuesto_PETU) AS impuesto_PETU,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(impuesto_IVA) AS impuesto_IVA,
        SUM(Impuestos_Total) AS Impuestos_Total,
        SUM(impuesto_PELR) AS impuesto_PELR,
        SUM(impuesto_PERN) AS impuesto_PERN
    FROM GroupedSales
    GROUP BY
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        cSalesPoint,
        Customer_DwID,
        Emission,
        Form,
        FreeGoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        npercentage,
        Pricing_DwID,
        PricingFormationRule_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Serie,
        Step_DwID,
        Territory_DwID,
        UnitValue
),
NewColumns AS (
    SELECT
        a.*,
        RIGHT(REPLICATE('0', 4) + a.Form, 4) + a.Serie + RIGHT(CAST(a.cSalesPoint AS VARCHAR), 4) + a.Invoice_DwID  as numero,
        RIGHT(a.Territory_DwID, 4) as codigo_ruta,
        'MC1' as Fuente,
        GETDATE() as Storeday,
        'Query' as Tabla,
        GETDATE() as Insert_date
    FROM GroupesSales2 a
    Where a.npercentage <> 0
),
FacturaRemitoQuery AS(
    select
    FACTURA.cIDBranchInvoice AS AGENCIA_DWID,
    ord.cIDOrder AS PEDIDO_GR, 
    ire.cSerie AS SERIE_GR, 
    MC1_GDescription.cDescription AS Form, 
    ire.cIDInvoice AS FACTURA_GR, 
    ire.xInvoiceRequestStatus AS STATUS_GR,
    convert(date, ire.dRequest ) as FECHA_EMISION_GR,
    orde.cIDOrderRef AS PEDIDO_FC,
    IREFC.cSerie AS SERIE_FC, 
    IREFC.cIDInvoice AS FACTURA_FC,
    IREFC.xInvoiceRequestStatus AS STATUS_FC,
    i.cSalesPoint,
    RIGHT(REPLICATE('0', 4) + MC1_GDescription.cDescription, 4) + ire.cSerie + RIGHT(CAST(i.cSalesPoint AS VARCHAR), 4) +  ire.cIDInvoice AS numero_remito,
    convert(date,irefc.drequest) as FECHA_EMISION_FC
    FROM CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_Order AS ORD
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_OrderExt AS ORDE 
    ON ORDE.cIDOrder = ord.cIDOrder
    and orde.mc1Enabled ='1'
    inner join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_OrderInvoice OINV
    on OINV.cIDOrder = ord.cIDOrder
    and oinv.mc1Enabled = '1'
    inner join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceRequestEmites IRE
    on ire.cIDInvoice = oinv.cIDInvoice
    and ire.cSerie = oinv.cSerie
    and ire.cForm = oinv.cForm
    and ire.cIDBranchInvoice = oinv.cIDBranchInvoice
    and ire.mc1Enabled ='1'
    and ire.cSerie = 'FR'
    left join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_OrderInvoice oinvFC
    on oinvFC.cIDOrder = orde.cIDOrderRef
    and oinvFC.mc1Enabled = '1'
    left join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_Invoice FACTURA
    on oinvFC.cIDInvoice = FACTURA.cIDInvoice
    and oinvFC.cSerie = FACTURA.cSerie
    and oinvFC.cForm = FACTURA.cForm
    and oinvFC.cIDBranchInvoice = FACTURA.cIDBranchInvoice
    and oinvFC.mc1Enabled ='1'
    left join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceRequestEmites IREFC
    on IREFC.cIDInvoice = FACTURA.cIDInvoice
    and IREFC.cSerie = FACTURA.cSerie
    and IREFC.cForm = FACTURA.cForm
    and IREFC.cIDBranchInvoice = FACTURA.cIDBranchInvoice
    and IREFC.mc1Enabled ='1'
    inner join (select *
    from CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceExt 
    ) i 
    on ire.cIDCompany = i.cIDCompany
    and ire.cIDInvoice = i.cIDInvoice
    and ire.cForm = i.cform
    and ire.cSerie = i.cSerie
    and ire.cIDBranchInvoice =i.cIDBranchInvoice
    AND i.mc1Enabled = 1
    left join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_GeneralDescription MC1_GDescription
    on ire.cForm = MC1_GDescription.cIDDomainType
    and MC1_GDescription.cDomainType = 'cForm'
    where 1=1
    and format(FACTURA.dEmission , 'yyyy-MM-dd') between '2024-03-16' and '2024-03-16' --informacion de un dia  

),
JoinWithRemitos AS (
    SELECT
        a.*,
        CASE
            WHEN b.numero_remito IS NOT NULL THEN b.numero_remito
            ELSE a.numero
        END as numero_remito
    FROM NewColumns a
    LEFT JOIN FacturaRemitoQuery b
    ON a.Invoice_DwID = b.FACTURA_FC
    AND a.BranchDwID = b.AGENCIA_DWID
    AND a.Order_DwID = b.PEDIDO_FC
),
VtoCAEQuery AS (
    SELECT DISTINCT a.cIDInvoice ,b.cSalesPoint 
    ,a.cSerie
    ,MC1_GDescription.cDescription Form
    ,c.cFiscalRegimeCode ,c.nFiscalSeqNumber 
    ,c.cFiscalNumberControl as CAI ,c.dEndFiscalNumberControl  as Fecha_VtoCAI, a.cIDCustomer,c.cDocument
    ,e.xIDPaymentCondition + cast(a.dEmission as int) Fecha_VtoRemito
    ,RIGHT(REPLICATE('0', 4) + MC1_GDescription.cDescription, 4) + a.cSerie+ RIGHT(CAST(b.cSalesPoint  AS VARCHAR), 4) + a.cIDInvoice as numero
    --top 100 a.*
    from CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_Invoice a
    inner join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceExt b 
    on a.cIDCompany = b.cIDCompany 
    and a.cIDInvoice = b.cIDInvoice 
    and a.cSerie = b.cSerie 
    and a.cForm = b.cForm 
    and a.mc1Enabled = 1
    and b.mc1Enabled = 1
    inner join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceNumberExt c 
    on a.cIDBranchInvoice = c.cIDBranchInvoice 
    and a.cSerie = c.cInvoiceSerie 
    and b.xDocType = c.xTypeDoc 
    and a.cIDCompany = c.cIDCompany 
    and UPPER(c.cDocument) in ('NF WEB','GUIA DE REMISION')
    and c.mc1Enabled = 1
    inner join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_OrderInvoice d
    on a.cIDCompany = d.cIDCompany 
    and a.cIDInvoice = d.cIDInvoice 
    and a.cSerie = d.cSerie 
    and a.cForm = d.cForm 
    and d.mc1Enabled = 1
    inner join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_Order e
    on e.cIDCompany = d.cIDCompany 
    and e.cidOrder = d.cidorder
    and d.mc1Enabled = 1
    left join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_GeneralDescription MC1_GDescription
    on a.cForm = MC1_GDescription.cIDDomainType
    and MC1_GDescription.cDomainType = 'cForm'
    where 1=1 
    --and a.xInvoiceType in ('ZOGR','ZOGRD')
    and format(a.dEmission , 'yyyy-MM-dd') between '2024-03-16' and '2024-03-16' --informacion de un dia  

    UNION 

    SELECT DISTINCT a.cIDInvoice ,b.cSalesPoint 
    ,a.cSerie
    ,MC1_GDescription.cDescription Form
    ,c.cFiscalRegimeCode ,c.nFiscalSeqNumber 
    ,c.cFiscalNumberControl as CAI ,c.dEndFiscalNumberControl  as Fecha_VtoCAI, a.cIDCustomer,c.cDocument
    --top 100 a.*
    ,e.xIDPaymentCondition + cast(a.dEmission as int) Fecha_VtoRemito
    ,RIGHT(REPLICATE('0', 4) + MC1_GDescription.cDescription, 4) + a.cSerie+ RIGHT(CAST(b.cSalesPoint  AS VARCHAR), 4) + a.cIDInvoice as numero
    from CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_Invoice a
    inner join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceExt b 
    on a.cIDCompany = b.cIDCompany 
    and a.cIDInvoice = b.cIDInvoice 
    and a.cSerie = b.cSerie 
    and a.cForm = b.cForm 
    and a.mc1Enabled = 1
    and b.mc1Enabled = 1
    inner join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_InvoiceNumberExt c 
    on a.cIDBranchInvoice = c.cIDBranchInvoice 
    and a.cSerie = c.cInvoiceSerie 
    and b.xDocType = c.xTypeDoc 
    and a.cIDCompany = c.cIDCompany 
    and UPPER(c.cDocument) in ('NF MOBILE')
    and a.cIDTerritory = c.cIDTerritory 
    and c.mc1Enabled = 1
    inner join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_OrderInvoice d
    on a.cIDCompany = d.cIDCompany 
    and a.cIDInvoice = d.cIDInvoice 
    and a.cSerie = d.cSerie 
    and a.cForm = d.cForm 
    and d.mc1Enabled = 1
    inner join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_Order e
    on e.cIDCompany = d.cIDCompany 
    and e.cidOrder = d.cidorder
    and d.mc1Enabled = 1
    left join CBRTM3XPOLCD22_LASC1.GRO_BA_RTM.VW_MC1_GeneralDescription MC1_GDescription
    on a.cForm = MC1_GDescription.cIDDomainType
    and MC1_GDescription.cDomainType = 'cForm'
    where 1=1 
    --and a.xInvoiceType in ('ZOGR','ZOGRD')
    and format(a.dEmission , 'yyyy-MM-dd') between '2024-03-16' and '2024-03-16' --informacion de un dia  
),
DeduplicatesVtoCAE AS (
	SELECT
        *
    FROM (
        SELECT
        a.*,
        ROW_NUMBER() OVER (PARTITION BY a.numero ORDER BY (SELECT NULL)) as rn
    FROM VtoCAEQuery a
    ) sub
    WHERE rn = 1
),
JoinWithVtoCAE AS (
    SELECT
        a.*,
        b.CAI,
        b.Fecha_VtoCAI
    FROM JoinWithRemitos a
    LEFT JOIN DeduplicatesVtoCAE b
    ON a.numero = b.numero
),
FinalJoinArgentinaMC1 AS (
    SELECT
        a.*,
        b.Fecha_VtoRemito
    FROM JoinWithVtoCAE a
    LEFT JOIN DeduplicatesVtoCAE b
    ON a.numero_remito = b.numero
)
SELECT * FROM FinalJoinArgentinaMC1